In [1]:
from DistMLIP.implementations.matgl import CHGNet_Dist, Potential_Dist, MolecularDynamics, Relaxer
import matgl
from pymatgen.core import Structure, Lattice
from pymatgen.io.ase import AseAtomsAdaptor

/data/kevinxhan/miniconda3/envs/distmlip/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize a CHGNet_Dist model (because CHGNet_Dist inherits from matgl's CHGNet class, model finetuning, training, etc. all work the exact same as in MatGL)
# However, we recommend using DistMLIP purely as an inference platform
model = CHGNet_Dist()

# Or load a previously trained CHGNet model
model = matgl.load_model("CHGNet-MatPES-PBE-2025.2.10-2.7M-PES").model

# Make a distributed version of the model
dist_model = CHGNet_Dist.from_existing(model)

# Enable distributed mode for 2 GPUs
dist_model.enable_distributed_mode(["cpu", "cpu"])

In [3]:
from ase.io import Trajectory
# Create atoms
bub_struct = Trajectory("/data/shared/Bowen_data/Kevin/gas_bubble.traj")

In [4]:
atoms = bub_struct[-1]

In [5]:
# Create Potential_Dist object, use 128 threads when creating graph structures
potential = Potential_Dist(model=dist_model, num_threads=128, calc_stresses=False)

In [6]:
# atoms = AseAtomsAdaptor().get_atoms(bub_struct)
a = potential(atoms)

get_subgraphs_fast time:First loop elapsed: 0.166753030 seconds
 57.72123825899325
Second loop elapsed: 53.188473105 seconds
Third loop elapsed: 0.018312382 seconds
Fourth loop elapsed: 0.030249282 seconds
TIMING: FPIS time: 56.620127
TIMING: wrapped frac to wrapped cartesian timing: 0.001310
Number of threads: 128
Cartesian maxes: 97.120901 97.258467 97.198514
Cartesian mins: 0.001316 0.005281 0.002782
Fractional longest dim max: 0.999994
Fractional longest dim min: 0.000024
Longest dim length in fractional coordinates: 0.999970
Checking collisions... 
Dim max: 0.999994. Dim min: 0.000024
First wall position: 0.500009
Created partition rule
Dim to use (based on cartesian coordinates): 1
Atom cutoff: 6.000000
Number of bonds within atom radius: 9134356
Number of bonds within bond radius: 1094244
USE_BOND_GRAPH IS 1
TIMING: Created partitions and partition rule: 0.000999670
First loop within assign_to_partitions_test_2: 0.003944700
Second loop in assign_to_partitions_test_2: 0.001003720

AssertionError: 

In [ ]:
#TODO (7.12.2025) LEFT OFF HERE: need to figure out why loop 2 is taking up so much time. I think it's a load balacning issue
# I've asked r1, we'll see what it says